In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets sentencepiece --quiet
# !pip install transformers --quiet
# !pip install accelerate --quiet
!pip install transformers[torch] --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import transformers
import numpy as np
import json
import os
from torch.utils.data import Dataset, random_split
import torch

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


# Load Pretrain Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,  MT5ForConditionalGeneration
pretrain_model_name = "google/mt5-base" # @param ["google/mt5-small", "google/mt5-base", "google/mt5-large", "google/mt5-xxl"]

tokenizer = AutoTokenizer.from_pretrained(pretrain_model_name, use_fast=False)


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
SPECIAL_TOKEN = ['<SYMBOL>','<ASPECT>','<OPINION>','<POS>','<NEG>','<NEU>']
special_tokens_dict = {'additional_special_tokens': SPECIAL_TOKEN}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrain_model_name)
# model = MT5ForConditionalGeneration.from_pretrained(pretrain_model_name)
model.resize_token_embeddings(len(tokenizer))

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Embedding(250106, 768)

# Load Dataset

dataset should look like this

|text (input)| label (output) |
|--|---|
|text | `<SYMBOL>` symbol_word `<ASPECT>` aspect_word `<OPINION>` opinion_word `<POS>\|<NEU>\|<NEG>`|



In [ ]:
dataset_folder = '/content/drive/MyDrive/Financial_news/text-to-label/dataset/'
dataset_filename = 'TTFJ_fillnote_label.csv'

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files=os.path.join(dataset_folder, dataset_filename))
# dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation': 'valid.csv', 'test': 'test.csv'})


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 14617
    })
})

In [ ]:
dataset['train'][0]

{'text': 'DOHOME รายได้ขายเพิ่ม! ดันกำไร Q2 โต 46% แตะ 436 ล้านบาท',
 'label': '<SYMBOL> DOHOME <ASPECT> รายได้ <OPINION> เพิ่ม <POS>'}

In [ ]:
# dataset['train'].train_test_split(test_size=0.2)

In [ ]:
def process_data_to_model_inputs(batch):
    # tokenize the text and labels
    input_encoding = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    batch["input_ids"] = input_encoding.input_ids
    batch["attention_mask"] = input_encoding.attention_mask
    output_encoding = tokenizer(batch["label"], padding="max_length", truncation=True, max_length=512)
    batch["labels"] = output_encoding.input_ids
    batch["decoder_attention_mask"] = output_encoding["attention_mask"]
    return batch




In [ ]:
# Tokenize the dataset
tokenized_dataset = dataset.map(process_data_to_model_inputs, batched=True)
# tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/14617 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 14617
    })
})

In [ ]:
# tokenized_dataset['train'][0]['attention_mask']

In [ ]:
tokenized_dataset['train'].train_test_split(test_size=0.2)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 11693
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 2924
    })
})

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(['text','label'])

In [ ]:
datacollator = lambda data: {
                              # 'input_ids': torch.stack([f[0] for f in data]),
                              'input_ids': data['input_ids'],
                              # 'attention_mask': torch.stack([f[1] for f in data]),
                              # 'labels': torch.stack([f[0] for f in data]),
                              'labels': data['labels'],
                              }

In [ ]:
def split_dataset(dataset, train_size=0.8, test_size=0.1, seed=42):

    # Calculate the validation size as the remaining portion not used by train and test
    val_size = 1 - (train_size + test_size)

    # Ensure the sizes sum up to 1
    if train_size + test_size + val_size != 1:
        raise ValueError("Train, test, and validation sizes must sum up to 1.")

    # Split the dataset into train, test, and validation sets
    dataset = dataset["train"].train_test_split(test_size=test_size + val_size, seed=seed)

    # Further split the test set into validation and test sets
    test_val_dataset = dataset['test'].train_test_split(test_size=val_size / (test_size + val_size), seed=seed)

    return {
        "train": dataset["train"],
        "validation": test_val_dataset["train"],
        "test": test_val_dataset["test"]
    }


In [ ]:
splits = split_dataset(tokenized_dataset)

In [ ]:
splits

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
     num_rows: 11693
 }),
 'validation': Dataset({
     features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
     num_rows: 1462
 }),
 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
     num_rows: 1462
 })}

# Fine-tuning

In [ ]:
project_root = '/content/drive/MyDrive/Financial_news/model/'
model_folder = 'test_model_attention_mask'
model_path = os.path.join(project_root, model_folder)
os.makedirs(model_path, exist_ok=True)
model_path

'/content/drive/MyDrive/Financial_news/model/test_model_attention_mask'

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results",          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=8,   # batch size per device during training
#     per_device_eval_batch_size=8,    # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
# )

training_args = TrainingArguments(output_dir=os.path.join(model_path,'model_output'),
                                  num_train_epochs=20,
                                  logging_steps=100,
                                  # save_steps=5000,
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4,
                                  warmup_steps=500,
                                  weight_decay=0.05,
                                  logging_dir=os.path.join(model_path,'logs'),
)

In [ ]:
def make_contiguous(model):
    for param in model.parameters():
        param.data = param.data.contiguous()

# เรียกใช้ make_contiguous ก่อนการเทรน
make_contiguous(model)


In [ ]:
!ls /content/drive/MyDrive/Financial_news/model/test_model_attention_mask/model_output/checkpoint-55000

config.json		optimizer.pt   scheduler.pt	   training_args.bin
generation_config.json	rng_state.pth  trainer_state.json


In [ ]:
# Make the model tensors contiguous
make_contiguous(model)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=splits["train"],
    eval_dataset=splits["validation"]
)

# Train the model, resuming from a specific checkpoint
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/Financial_news/model/test_model_attention_mask/model_output/checkpoint-54000")


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3098: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on

Step,Training Loss
54100,0.003300
54200,0.003100
54300,0.003300
54400,0.003000
54500,0.003300
54600,0.003100
54700,0.003300
54800,0.003000
54900,0.003100
55000,0.003000


TrainOutput(global_step=58480, training_loss=0.00023979120537455202, metrics={'train_runtime': 2445.7879, 'train_samples_per_second': 95.617, 'train_steps_per_second': 23.91, 'total_flos': 2.8040561789239296e+17, 'train_loss': 0.00023979120537455202, 'epoch': 20.0})

In [ ]:
# # Make the model tensors contiguous
# make_contiguous(model)

# # Initialize Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=splits["train"],
#     eval_dataset=splits["validation"],
# )

# # Train the model
# trainer.train()


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=splits["train"],
    eval_dataset=splits["validation"],
    # data_collator=datacollator,
    # data_collector=lambda data: {'input_ids':

    # }
)


# Train the model
trainer.train()

In [ ]:
# Save the model and tokenizer
saved_model_path = os.path.join(model_path, 'save_model')
model.save_pretrained(saved_model_path)
tokenizer.save_pretrained(saved_model_path)

('/content/drive/MyDrive/Financial_news/model/test_model_attention_mask/save_model/tokenizer_config.json',
 '/content/drive/MyDrive/Financial_news/model/test_model_attention_mask/save_model/special_tokens_map.json',
 '/content/drive/MyDrive/Financial_news/model/test_model_attention_mask/save_model/spiece.model',
 '/content/drive/MyDrive/Financial_news/model/test_model_attention_mask/save_model/added_tokens.json')

In [ ]:
model.save_pretrained(saved_model_path)


In [ ]:
import os

# ตรวจสอบไฟล์ในโฟลเดอร์ save_model
saved_model_path = '/content/drive/MyDrive/Financial_news/model/test_model_attention_mask/save_model'
os.listdir(saved_model_path)


['config.json',
 'generation_config.json',
 'model.safetensors',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'added_tokens.json',
 'spiece.model']

In [ ]:
# saved_model_path = '/content/drive/MyDrive/Financial_news/model/test_model_attention_mask/save_model2'

# # Save the model and tokenizer
# model.save_pretrained(saved_model_path)
# tokenizer.save_pretrained(saved_model_path)

# # ตรวจสอบไฟล์ในโฟลเดอร์อีกครั้ง
# os.listdir(saved_model_path)


In [ ]:
dataset['train'][0]

{'text': 'DOHOME รายได้ขายเพิ่ม! ดันกำไร Q2 โต 46% แตะ 436 ล้านบาท',
 'label': '<SYMBOL> DOHOME <ASPECT> รายได้ <OPINION> เพิ่ม <POS>'}

In [ ]:
def generate_text(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    input_ids = input_ids.to(device)
    model.to(device)

    outputs = model.generate(input_ids, num_beams=1, do_sample=False, max_length=30)
    return tokenizer.decode(outputs[0])

# Example
input_text = "ICHI โบรกแนะซื้อเป้า 17.20 บ. ชี้หุ้นปลอดภัยนโยบายการเมือง-เอลนีโญยาว 3 ปีหนุน"
print(generate_text(input_text))


<pad> <SYMBOL> ICHI <ASPECT> โบรกแนะ <OPINION> ซื้อ <POS> </s>


In [ ]:
# tokenizer.decode(splits['test'][0]['labels'], skip_special_tokens=True)
tokenizer.decode(tokenizer('<SYMBOL> ICHI <ASPECT> หุ้น <OPINION> ปลอดภัย <POS>').input_ids)

'<SYMBOL> ICHI <ASPECT> หุ้น <OPINION> ปลอดภัย <POS> </s>'

```python
model_name/
│
├── model_output/             # output_dir from TrainingArguments
│   ├── checkpoint-500/       # Example checkpoint folder (after 500 steps)
│   ├── checkpoint-1000/      # Another checkpoint folder
│   └── ...                   # Other checkpoints
│
├── logs/                     # logging_dir from TrainingArguments
│   ├── run-20240926_000000/  # Log files for a specific run
│   └── ...                   # Other log files
│
└── save_model/              # Model saved after fine-tuning
    ├── config.json           # Model configuration
    ├── pytorch_model.bin     # Model weights
    ├── tokenizer_config.json # Tokenizer configuration
    ├── spiece.model          # SentencePiece model file (for some tokenizers)
    └── ...                   # Other tokenizer files


```